In [ ]:
using CSV, DataFrames
using CairoMakie
using GLMakie
using GeometryBasics
using DataStructures
using ProgressMeter

# Reading the text file assuming it's space-delimited
df = CSV.read("dataSiO2-1.dat", DataFrame; delim=' ', ignorerepeated=true,header=false)

# Display the first few rows
first(df, 5)


GLMakie.activate!()
aspect=(1, 1, 1)
perspectiveness=0.5
# the figure
fig = Figure(; size=(600, 600))
ax2 = Axis3(fig[1, 1]; aspect, perspectiveness)
meshscatter!(ax2, x, y, z; markersize=type/2, color=type)

fig




In [ ]:
rc = 2.5
# Define a function to compute and store points within a cutoff distance
function find_nearby_points(x, y, z, n, rc)
    # Initialize a dictionary to store i as key and corresponding j's as values
    A = Dict{Int, Vector{Int}}()
    # Loop over all points
    @showprogress for i = 1:n-1
        # Initialize an empty array to collect all j's for this i
        js = []
        for j = i+1:n
            # Calculate the distance between points (i, j)
            d = √((x[i]-x[j])^2 + (y[i]-y[j])^2 + (z[i]-z[j])^2)
            if d < rc
                # Add j to the list of neighbors for this i
                push!(js, j)
            end
        end
        # Save the results in the dictionary if any j's were found
        if !isempty(js)
            A[i] = js
        end
    end
    sorted_A = SortedDict(A)
    return A
end
A = find_nearby_points(x, y, z, n, rc);

In [ ]:


GLMakie.activate!()
aspect=:data
perspectiveness=0.5
# the figure
fig = Figure(; size=(600, 600))
ax1 = Axis3(fig[1, 1]; aspect, perspectiveness)
meshscatter!(ax1, x, y, z; markersize=type/2, color=type)


@showprogress for i = 1:n
    if haskey(A, i)
        point31 = Point(x[i], y[i], z[i])
        for j in A[i]
            point32 = Point(x[j], y[j], z[j])
            cyl1 = GeometryBasics.Cylinder(point31, (point31+point32)/2, 0.1)
            cyl2 = GeometryBasics.Cylinder((point31+point32)/2, point32, 0.1)
            mesh!(ax1, cyl1, color =:blue)
            mesh!(ax1, cyl2, color =:red)
        end
    end
end

fig